# Toronto Clustering Lab
## Submission for question 3 of 3

Start from previous submission state.

In [1]:
%run ./"IBM Capstone Project - Toronto Clustering Lab 2 of 3.ipynb"

Requirement already up-to-date: lxml in /opt/home_local/lib/python3.8/site-packages (4.6.2)
Requirement already up-to-date: html5lib in /opt/home_local/lib/python3.8/site-packages (1.1)
Requirement already up-to-date: beautifulsoup4 in /opt/home_local/lib/python3.8/site-packages (4.9.3)
Web scraping with pandas installed and imported successfully.
Scraped 180 rows successfully.
103
Geospatial information downloaded successfully.


## 1. Download and Explore Dataset

1.1. I will only explore the neighbourhoods in Toronto.

In [3]:
toronto_data = canada_postal_codes[canada_postal_codes.Borough.str.contains('Toronto')]

1.2. Prepare all the dependencies for this exercise.

In [4]:
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim

import requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported successfully.')

Libraries imported successfully.


1.3. Count totals from the **Toronto** dataset.

In [5]:
print('Toronto has {} boroughs and {} neighborhoods.'.format(
        len(toronto_data['Borough'].unique()),
        toronto_data.shape[0]
    )
)

Toronto has 4 boroughs and 39 neighborhoods.


1.4. Get coordinates of Toronto.

In [6]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


1.5. Visualize Toronto with the neighbourhoods.

In [7]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

1.6. I am going to start utilizing the Foursquare API to explore the neighborhoods and segment them. Here, I will define the credentials.

In [9]:
CLIENT_ID = '5PB2NYAAGTUF0MMQSQHLEIEC3C25VCDUNAAJUQWNR4KLHJBZ' # your Foursquare ID
CLIENT_SECRET = 'x' # your Foursquare Secret
ACCESS_TOKEN = 'x' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: 5PB2NYAAGTUF0MMQSQHLEIEC3C25VCDUNAAJUQWNR4KLHJBZ


1.7. Let's explore the first neighborhood in my dataframe.

In [10]:
neighborhood_name = toronto_data['Neighbourhood'].values[0]
print(neighborhood_name)

Regent Park, Harbourfront


1.8. Get the neighborhood's latitude and longitude values.

In [11]:
neighborhood_latitude = toronto_data['Latitude'].values[0]
neighborhood_longitude = toronto_data['Longitude'].values[0]

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


1.9. Create the GET request URL to obtain the top 100 venues that are in the neighbourhood within a radius of 500 meters.

In [12]:
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

1.10. Send the GET request and examine the results.

In [14]:
results = requests.get(url).json()
len(results)

2

1.11. Prepare a function that will help me to get categories.

In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

1.12. Clean the json and structure it into a pandas dataframe.

In [17]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
nearby_venues.head()

30 venues were returned by Foursquare.


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
4,Body Blitz Spa East,Spa,43.654735,-79.359874


## 2. Explore Neighborhoods in Toronto

2.1. Create a function to repeat the same process to all the neighborhoods in Toronto.

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, 
            lat, lng, radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']

        venues_list.append([(
            name, 
            lat, lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']

    return(nearby_venues)

2.2. Run the above function on each neighborhood and create a new dataframe.

In [20]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'], latitudes=toronto_data['Latitude'], longitudes=toronto_data['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

2.3. Check how many unique categories can be curated from all the returned venues and how many venues were returned for each neighborhood.

In [26]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
toronto_venues.groupby('Neighborhood').count()

There are 196 uniques categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",15,15,15,15,15,15
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,30,30,30,30,30,30
Christie,16,16,16,16,16,16
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
Davisville,30,30,30,30,30,30


## 3. Analyze Each Neighborhood

3.1. Check the new size after adding Neighborhood column again.

In [28]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe, to the first position
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.shape

(856, 196)

3.2. Print with the 5 most common venues.

In [30]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.10
1         Cheese Shop  0.07
2      Farmers Market  0.07
3        Cocktail Bar  0.07
4  Seafood Restaurant  0.07


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3    Intersection  0.05
4   Burrito Place  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0  Light Rail Station  0.13
1       Garden Center  0.07
2             Brewery  0.07
3          Comic Shop  0.07
4      Farmers Market  0.07


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0       Airport Lounge  0.13
1      Airport Service  0.13
2     Airport Terminal  0.13
3  Rental Car Location  0.07
4      Harbor / Marina  0.07


----Central Bay Street----
                venue  freq
0 

3.3. Create a sorted dataframe with the top 10 venues.

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cheese Shop,Farmers Market,Cocktail Bar,Seafood Restaurant,Beer Bar,French Restaurant,Bistro,Fountain,Fish Market
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Intersection,Burrito Place,Convenience Store,Stadium,Bar,Bakery,Restaurant
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Garden Center,Brewery,Comic Shop,Farmers Market,Fast Food Restaurant,Burrito Place,Auto Workshop,Skate Park,Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Rental Car Location,Harbor / Marina,Bar,Sculpture Garden,Coffee Shop,Boat or Ferry,Airport Gate
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Yoga Studio,Sandwich Place,Park,Spa,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant


## 4. Cluster Neighborhoods

4.1. Run k-means to cluster the neighborhood into 5 clusters.

In [42]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

4.2. Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [45]:
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_data

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
toronto_merged.head()

,PostalCode,Borough,Neighbourhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636,1,Coffee Shop,Park,Bakery,Breakfast Spot,Gym / Fitness Center,Pub,Dessert Shop,Chocolate Shop,Restaurant,French Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M7A,43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Yoga Studio,College Auditorium,Beer Bar,Smoothie Shop,Burrito Place,Café,Sandwich Place,Portuguese Restaurant
9,M5B,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937,1,Café,Theater,Coffee Shop,Bookstore,Tanning Salon,Burger Joint,Shopping Mall,Mexican Restaurant,Sandwich Place,Sporting Goods Shop
15,M5C,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418,1,Café,Farmers Market,Coffee Shop,Gastropub,Cosmetics Shop,Thai Restaurant,Restaurant,Food Truck,Fountain,New American Restaurant
19,M4E,East Toronto,The Beaches,M4E,43.676357,-79.293031,1,Trail,Health Food Store,Pub,Yoga Studio,Monument / Landmark,Liquor Store,Lounge,Market,Martial Arts School,Men's Store


4.3. Visualize the resulting clusters.

In [47]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters